In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [2]:
import json
import pandas as pd

data = []

with open('/content/News_Category_Dataset_v3.json', 'r', encoding='utf-8') as f:
    for line in f:
        try:
            data.append(json.loads(line))
        except:
            continue

df = pd.DataFrame(data)
print(df.shape)
print(df.columns)



(164499, 6)
Index(['link', 'headline', 'category', 'short_description', 'authors', 'date'], dtype='object')


In [3]:
df.shape

(164499, 6)

In [4]:
df.head()

,link,headline,category,short_description,authors,date
0,https://www.huffpost.com/entry/covid-boosters-...,Over 4 Million Americans Roll Up Sleeves For O...,U.S. NEWS,Health experts said it is too early to predict...,"Carla K. Johnson, AP",2022-09-23
1,https://www.huffpost.com/entry/american-airlin...,"American Airlines Flyer Charged, Banned For Li...",U.S. NEWS,He was subdued by passengers and crew when he ...,Mary Papenfuss,2022-09-23
2,https://www.huffpost.com/entry/funniest-tweets...,23 Of The Funniest Tweets About Cats And Dogs ...,COMEDY,"""Until you have a dog you don't understand wha...",Elyse Wanshel,2022-09-23
3,https://www.huffpost.com/entry/funniest-parent...,The Funniest Tweets From Parents This Week (Se...,PARENTING,"""Accidentally put grown-up toothpaste on my to...",Caroline Bologna,2022-09-23
4,https://www.huffpost.com/entry/amy-cooper-lose...,Woman Who Called Cops On Black Bird-Watcher Lo...,U.S. NEWS,Amy Cooper accused investment firm Franklin Te...,Nina Golgowski,2022-09-22


In [5]:
df.columns

Index(['link', 'headline', 'category', 'short_description', 'authors', 'date'], dtype='object')

In [6]:
df['text']=df['headline'].fillna('')+' '+df['short_description'].fillna('')

In [7]:
Feature=df['text']
Target=df['category']

In [8]:
Feature.head(),Target.head()

(0    Over 4 Million Americans Roll Up Sleeves For O...
 1    American Airlines Flyer Charged, Banned For Li...
 2    23 Of The Funniest Tweets About Cats And Dogs ...
 3    The Funniest Tweets From Parents This Week (Se...
 4    Woman Who Called Cops On Black Bird-Watcher Lo...
 Name: text, dtype: object,
 0    U.S. NEWS
 1    U.S. NEWS
 2       COMEDY
 3    PARENTING
 4    U.S. NEWS
 Name: category, dtype: object)

In [9]:
import re
def clean_text(text):
  text=text.lower()
  text=re.sub(r'[^a-z\s]',' ',text)
  text = re.sub(r'\s+', ' ', text).strip()
  return text

In [10]:
Feature_clean=Feature.apply(clean_text)

In [11]:
Feature_clean.head()

,text
0,over million americans roll up sleeves for omi...
1,american airlines flyer charged banned for lif...
2,of the funniest tweets about cats and dogs thi...
3,the funniest tweets from parents this week sep...
4,woman who called cops on black bird watcher lo...


In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [13]:
tfidf=TfidfVectorizer(stop_words='english',max_features=50000,ngram_range=(1,2))
Feature_tfidf=tfidf.fit_transform(Feature_clean)

In [14]:
Feature_tfidf[0:]

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 2716175 stored elements and shape (164499, 50000)>

In [15]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(Feature_tfidf,Target,test_size=0.2,stratify=Target)
X_train.shape, X_test.shape


((131599, 50000), (32900, 50000))

In [16]:
from sklearn.linear_model import LogisticRegression
lr=LogisticRegression(max_iter=1000,n_jobs=-1)
model=lr.fit(X_train,Y_train)

In [17]:
Y_pred=model.predict(X_test)

In [18]:
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
print("Accuracy:", accuracy_score(Y_test, Y_pred))
print(classification_report(Y_test, Y_pred))


Accuracy: 0.5735866261398176
                precision    recall  f1-score   support

          ARTS       0.46      0.26      0.33       302
ARTS & CULTURE       0.46      0.14      0.21       268
  BLACK VOICES       0.58      0.34      0.43       827
      BUSINESS       0.52      0.48      0.50       987
       COLLEGE       0.54      0.38      0.44       229
        COMEDY       0.61      0.35      0.44       917
         CRIME       0.57      0.53      0.55       656
CULTURE & ARTS       1.00      0.02      0.04        94
       DIVORCE       0.82      0.58      0.68       306
     EDUCATION       0.56      0.37      0.45       203
 ENTERTAINMENT       0.55      0.82      0.66      3295
   ENVIRONMENT       0.92      0.07      0.14       150
         FIFTY       0.61      0.15      0.25       280
  FOOD & DRINK       0.52      0.59      0.55       624
     GOOD NEWS       0.54      0.15      0.24       280
         GREEN       0.44      0.37      0.40       524
HEALTHY LIVING    

In [19]:
X_train_small = X_train[:60000]
Y_train_small = Y_train[:60000]
from sklearn.svm import LinearSVC

svm = LinearSVC(max_iter=2000)

model1=svm.fit(X_train_small, Y_train_small)



In [21]:
Y_pred1=model1.predict(X_test)
print("Accuracy:", accuracy_score(Y_test, Y_pred1))
print(classification_report(Y_test, Y_pred1))


Accuracy: 0.5608510638297872
                precision    recall  f1-score   support

          ARTS       0.38      0.32      0.35       302
ARTS & CULTURE       0.30      0.14      0.19       268
  BLACK VOICES       0.49      0.40      0.44       827
      BUSINESS       0.48      0.45      0.46       987
       COLLEGE       0.52      0.45      0.48       229
        COMEDY       0.46      0.36      0.41       917
         CRIME       0.53      0.54      0.54       656
CULTURE & ARTS       0.24      0.06      0.10        94
       DIVORCE       0.76      0.67      0.71       306
     EDUCATION       0.48      0.40      0.44       203
 ENTERTAINMENT       0.61      0.75      0.68      3295
   ENVIRONMENT       0.36      0.14      0.20       150
         FIFTY       0.38      0.21      0.27       280
  FOOD & DRINK       0.51      0.56      0.54       624
     GOOD NEWS       0.39      0.19      0.26       280
         GREEN       0.40      0.39      0.39       524
HEALTHY LIVING    